In [1]:
import cv2
from deepface import DeepFace

# Load the Haar Cascade for face detection
faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Open the webcam
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    cap = cv2.VideoCapture(1)
if not cap.isOpened():
    raise IOError("Cannot open webcam")

# Initialize the last emotion variable
last_emotion = "No face detected"

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        break

    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = faceCascade.detectMultiScale(gray, 1.1, 4)
    for (x, y, w, h) in faces:
        # Draw rectangle around the detected face
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        try:
            # Crop the face region for DeepFace analysis
            roi_color = frame[y:y+h, x:x+w]
            result = DeepFace.analyze(roi_color, actions=['emotion'], enforce_detection=False)
            last_emotion = result[0]['dominant_emotion']
        except Exception as e:
            print(f"DeepFace analyze error: {e}")

    # Display the emotion on the frame
    font = cv2.FONT_HERSHEY_SIMPLEX
    fontScale = 2
    thickness = 4
    color = (255, 255, 255)
    cv2.putText(frame,
                last_emotion,
                (50, 50),
                font,
                fontScale,
                color,
                thickness,
                cv2.LINE_AA)

    # Display the resulting frame
    cv2.imshow('Demo Video', frame)

    # Press 'q' to quit
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

# Release the webcam and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()
